# Reuters Graph version

### Sources
- [Philipp tutorial](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial7/GNN_overview.html)
- [pytorch-geometric](https://pytorch-geometric.readthedocs.io/en/latest/)
- [nltk docs](https://www.nltk.org/book/ch02.html)

In [1]:
import torch
import numpy as np

from data_prep.reuters_graph import R8Graph, R52Graph

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

[nltk_data] Downloading package reuters to /home/mat/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [2]:
r8 = R8Graph(device)

Prepare Reuters dataset
Compute tf.idf
Compute PMI scores
Generate edges
Generate masks
Generate feature matrix
Features mtx is 0.5672018560000001 GBs in size


In [3]:
print(np.unique(r8.data.y[r8.data.train_mask].cpu(), return_counts=True))
print(np.unique(r8.data.y[r8.data.val_mask].cpu(), return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6, 7]), array([259,  37, 482,  20,  27,  32,  27, 116]))
(array([0, 1, 2, 3, 4, 5, 6, 7]), array([25,  7, 50,  1,  6,  3,  2,  6]))


In [4]:
print(sum(r8.data.val_mask * r8.data.train_mask * r8.data.test_mask))
print(sum(r8.data.train_mask))
print(sum(r8.data.val_mask))
print(sum(r8.data.test_mask))

tensor(0, device='cuda:0')
tensor(1000, device='cuda:0')
tensor(100, device='cuda:0')
tensor(100, device='cuda:0')


In [5]:
# GraphConv, GATConv
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(len(r8.iton), 200)
        self.conv2 = GCNConv(200, 8)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

def eval(model, data, mask):
    _, pred = model(data).max(dim=1)
    #print(pred[mask])
    #print(data.y[mask])
    correct = pred[mask].eq(data.y[mask]).sum().item()
    acc = correct / mask.sum()
    print('Accuracy: {:.4f}'.format(acc))

In [6]:
model = Net()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)#, weight_decay=5e-4)
r8.data.to(device)

Data(edge_attr=[1191234], edge_index=[2, 1191234], test_mask=[11908], train_mask=[11908], val_mask=[11908], x=[11908, 11908], y=[11908])

In [7]:
from tqdm.notebook import tqdm

model.train()
for epoch in tqdm(range(40)):
    optimizer.zero_grad()
    out = model(r8.data)
    # We might want to use the "weight" parameter for the loss with unbalanced dataset
    # since with a low learning rate the model just assigns every doc to class "earn"
    # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
    loss = F.cross_entropy(out[r8.data.train_mask], r8.data.y[r8.data.train_mask])
    print('Loss:', loss.item())
    loss.backward()
    optimizer.step()
    print(np.unique(out[r8.data.train_mask].max(dim=1)[1].detach().cpu().numpy(), return_counts=True))
    #print(np.unique(r8.data.y[r8.data.train_mask].detach().cpu().numpy(), return_counts=True))
    eval(model, r8.data, r8.data.val_mask)

Loss: 2.0794546604156494
(array([0, 1, 2, 3, 4, 5, 6, 7]), array([  8, 669,  94,  99,   1,  26,  81,  22]))
Accuracy: 0.5000
Loss: 1.5738054513931274
(array([2]), array([1000]))
Accuracy: 0.5000
Loss: 1.3624327182769775
(array([2]), array([1000]))
Accuracy: 0.5000
Loss: 1.299436092376709
(array([2]), array([1000]))
Accuracy: 0.6300
Loss: 1.1411274671554565
(array([0, 2]), array([273, 727]))
Accuracy: 0.6700
Loss: 0.9985102415084839
(array([0, 2]), array([436, 564]))
Accuracy: 0.6800
Loss: 0.8667310476303101
(array([0, 2, 7]), array([421, 578,   1]))
Accuracy: 0.7000
Loss: 0.7822335958480835
(array([0, 2, 7]), array([408, 591,   1]))
Accuracy: 0.6900
Loss: 0.7161081433296204
(array([0, 2, 7]), array([429, 570,   1]))
Accuracy: 0.6900
Loss: 0.660102367401123
(array([0, 2, 7]), array([453, 544,   3]))
Accuracy: 0.6900
Loss: 0.6130087971687317
(array([0, 2, 7]), array([431, 541,  28]))
Accuracy: 0.7400
Loss: 0.5579763054847717
(array([0, 2, 7]), array([327, 537, 136]))
Accuracy: 0.7400
Los

In [8]:
eval(model, r8.data, r8.data.test_mask)

Accuracy: 0.8500
